In [1]:
import cv2
import os
import pandas as pd
from PIL import Image
import math
import numpy as np
from random import randint

### Background Images

In [ ]:
req_size = 224
source = r'D:\Assignment 15A\Raw Images\01. Scene Images'
destination = r'D:\Assignment 15A\Resized Images\01. Backgrounds'

os.chdir(source)
files = os.listdir()
file_map = pd.DataFrame({'Old Name':os.listdir(),
                         'New Name':['bg'+'{:03}'.format(i)+'.jpg'  for i in list(range(1,len(os.listdir())+1))]})

file_map['Old Name'].apply(lambda x:x.split('.')[len(x.split('.'))-1]).unique()

for i in list(range(0,file_map.shape[0])):
    print(i)
    os.chdir(source)
    im = Image.open(source+'\\'+file_map.loc[i,'Old Name']).convert('RGB')
    os.chdir(destination)
    if (im.size[0]/im.size[1])>1:
        factor = im.size[1]/req_size
    else:
        factor = im.size[0]/req_size
    
    im = im.resize((math.floor(im.size[0]/factor),math.floor(im.size[1]/factor)))
        
    if im.size[0]>im.size[1]:
        dims = (math.floor(im.size[0]/2)-(req_size/2),0,math.floor(im.size[0]/2)+(req_size/2),req_size)
    else:
        dims = (0,math.floor(im.size[1])-req_size,req_size,math.floor(im.size[1]))
            
    im = im.crop(dims)
    
    im.save(file_map.loc[i,'New Name'],'jpeg')

file_map.to_csv(r'D:\Assignment 15A\Raw Images\Background Mapper.csv')

### Foreground Images

In [ ]:
req_size = 56
source = r'D:\Assignment 15A\Raw Images\02. Objects'
destination = r'D:\Assignment 15A\Resized Images\02. Foregrounds'

os.chdir(source)
files = os.listdir()
file_map = pd.DataFrame({'Old Name':os.listdir(),
                         'New Name':['fg'+'{:03}'.format(i)+'.png'  for i in list(range(1,len(os.listdir())+1))],
                         'Flip Name':['fg'+'{:03}'.format(100+i)+'.png'  for i in list(range(1,len(os.listdir())+1))]})

for i in list(range(0,file_map.shape[0])):
    print("------------------")
    print(i)    
    os.chdir(source)
    im = cv2.imread(source+'\\'+file_map.loc[i,'Old Name'], cv2.IMREAD_UNCHANGED)
    print(im.shape)
    os.chdir(destination)
    if (im.shape[0]<im.shape[1]):
        factor = im.shape[0]/req_size
    else:
        factor = im.shape[1]/req_size
        
    dim = (math.floor(im.shape[1]/factor),math.floor(im.shape[0]/factor))
    im = cv2.resize(im,dim, interpolation = cv2.INTER_AREA)
    flipHorizontal = cv2.flip(im, 1)
    print(im.shape)
    cv2.imwrite(file_map.loc[i,'New Name'],im)
    cv2.imwrite(file_map.loc[i,'Flip Name'],flipHorizontal)

file_map.to_csv(r'D:\Assignment 15A\Raw Images\Foreground Mapper.csv')


### Masking Images

In [ ]:
source = r'D:\Assignment 15A\Resized Images\02. Foregrounds'
destination = r'D:\Assignment 15A\Resized Images\03. Masks'

os.chdir(source)
files = os.listdir()
file_map = pd.DataFrame({'Old Name':os.listdir(),
                         'New Name':['mk'+'{:03}'.format(i)+'.png'  for i in list(range(1,len(os.listdir())+1))]})

for i in list(range(0,file_map.shape[0])):
    print("-----------------------------")
    print(i)    
    os.chdir(source)
    im = Image.open(source+'\\'+file_map.loc[i,'Old Name']).convert('RGB')
    os.chdir(destination)    
    # im = cv2.imwrite('sample.jpg',im)
    # im = cv2.imread('sample.jpg')
    im = np.array(im)
    im[im != 0] = 255
    pil_img = Image.fromarray(im)
    pil_img.save(file_map.loc[i,'New Name'])

file_map.to_csv(r'D:\Assignment 15A\Raw Images\Mask Mapper.csv')


### Overlaid Images and Masks

In [ ]:
background = r'D:\Assignment 15A\Resized Images\01. Backgrounds'
foreground = r'D:\Assignment 15A\Resized Images\02. Foregrounds'
masks = r'D:\Assignment 15A\Resized Images\03. Masks'
overlaid = r'D:\Assignment 15A\Resized Images\04. Overlaid'
o_masks = r'D:\Assignment 15A\Resized Images\05. Overlaid Masks'


ims_b = os.listdir(background)
ims_f = os.listdir(foreground)
ims_m = os.listdir(masks)

os.chdir(background)
bg_shape = 224
black = np.zeros((224,224,3))
black =  Image.fromarray(black, 'RGB')

for i in ims_b:
    print("+++++++++++++++++++++++++++++++++++++")
    print(i)
    os.chdir(background)
    bg_ = Image.open(i)
    bg_name = i.split('.')[0]+'_'
    for j in list(range(0,200)):
        print("------------------------------------")
        os.chdir(foreground)
        fg_ = Image.open(ims_f[j])
        os.chdir(masks)
        mk_ = Image.open(ims_m[j])
        max_x = 224-fg_.size[0]
        max_y = 224-fg_.size[1]
        fg_name = ims_f[j].split('.')[0]+'_'
        ms_name = ims_m[j].split('.')[0]+'_'
        xy_cords = [(randint(0, max_x),randint(0, max_y)) for p in range(0, 20)]
        
        imge = 0
        for xy in xy_cords:
            print(xy)
            background_O = bg_.copy()
            foreground_O = fg_.copy()
            black_O = black.copy()
            mask_O = mk_.copy().convert("RGBA")
            imge = imge+1
            
            background_O.paste(foreground_O, xy, foreground_O)
            name = bg_name+fg_name+'{:02}'.format(imge)+'.jpg'
            os.chdir(overlaid)
            background_O.save(name,"jpeg")
            
            os.chdir(o_masks)
            name = bg_name+ms_name+'{:02}'.format(imge)+'.jpg'
            black_O.paste(mask_O, xy, mask_O)
            black_O.save(name,"jpeg")

### Depth Estimation

Link to Repository: https://github.com/jai2shan/Depth-Estimation-PyTorch

Used test_jay.py to estimate the depths in the images